In [6]:
import mlflow

mlflow.set_tracking_uri("file:///C:/Users/titli/Documents/sms-spam-dvc/mlruns")
mlflow.set_experiment("sms_spam_benchmarks")


c:\Users\titli\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/02/12 00:33:05 INFO mlflow.tracking.fluent: Experiment with name 'sms_spam_benchmarks' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/titli/Documents/sms-spam-dvc/mlruns/368038252911427570', creation_time=1770836585445, experiment_id='368038252911427570', last_update_time=1770836585445, lifecycle_stage='active', name='sms_spam_benchmarks', tags={}>

In [8]:
import pandas as pd

train_df = pd.read_csv("data/train.csv")
test_df  = pd.read_csv("data/test.csv")


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english")

X_train = vectorizer.fit_transform(train_df["text"])
y_train = train_df["label"]

X_test = vectorizer.transform(test_df["text"])
y_test = test_df["label"]


In [14]:
from sklearn.metrics import average_precision_score

def compute_aucpr(model, X, y):
    """
    Compute AUCPR (Average Precision Score)

    Works for:
    - Logistic Regression (decision_function)
    - Linear SVM (decision_function)
    - Naive Bayes (predict_proba)
    """

    if hasattr(model, "decision_function"):
        scores = model.decision_function(X)

    elif hasattr(model, "predict_proba"):
        scores = model.predict_proba(X)[:, 1]

    else:
        raise ValueError("Model does not support scoring for AUCPR")

    return average_precision_score(y, scores)



In [15]:
import mlflow
import mlflow.sklearn

def run_experiment(model, name):

    with mlflow.start_run(run_name=name):

        model.fit(X_train, y_train)

        aucpr = compute_aucpr(model, X_test, y_test)

        mlflow.log_metric("AUCPR", aucpr)
        mlflow.sklearn.log_model(model, "model")

        print(name, "AUCPR =", aucpr)

        return aucpr


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "NaiveBayes": MultinomialNB(),
    "LinearSVM": LinearSVC(dual="auto")
}

results = {}

for name, model in models.items():
    results[name] = run_experiment(model, name)


2026/02/12 00:50:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\titli\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/12 00:50:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


LogisticRegression AUCPR = 0.9638042020737022


c:\Users\titli\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
2026/02/12 00:50:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


NaiveBayes AUCPR = 0.9679638068159969


c:\Users\titli\AppData\Local\Programs\Python\Python312\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


LinearSVM AUCPR = 0.967608503553906


In [17]:
def run_experiment(model, name):

    with mlflow.start_run(run_name=name):

        model.fit(X_train, y_train)

        aucpr = compute_aucpr(model, X_test, y_test)

        mlflow.log_metric("AUCPR", aucpr)

        # Log model
        mlflow.sklearn.log_model(model, name="model")

        # Register model in MLflow Model Registry
        mlflow.register_model(
            model_uri=f"runs:/{mlflow.active_run().info.run_id}/model",
            name=name
        )

        print(name, "AUCPR =", aucpr)

        return aucpr


In [18]:
print("\nFinal Model Comparison (AUCPR):\n")

for model, score in results.items():
    print(f"{model}: AUCPR = {score:.4f}")

best_model = max(results, key=results.get)
print("\nBest model selected:", best_model)



Final Model Comparison (AUCPR):

LogisticRegression: AUCPR = 0.9638
NaiveBayes: AUCPR = 0.9680
LinearSVM: AUCPR = 0.9676

Best model selected: NaiveBayes
